## Capstone Project

# Using machine learning techniques to identify the strongest exoplanet candidates from the cumulative Kepler Objects of Interest Activity Table

# 1. The problem

## 1.1 Context

In March 2009, the NASA launched the Kepler space telescope. Its mission was to discover earth-size exoplanets in or near habitable zones. Equiped with a photometer, Kepler continuously monitored the brightness of approximately 150'000 stars in a portion of Earth's region of the Milky Way. During its nine and half year of lifespan, Kepler has detected 2'662 planets. Planets are indirectly detected by the dimming of the brigthness when they cross in front of their host star. De facto, Kepler was able to detect only planets whose orbits are seen edge-on from Earth. 

These registred brigthness dimmings can be caused by exoplanet transits but also by other phenomena like binary stars and secondary eclipses, when the planet passes behind its host star. The analysis of these events 
called **Threshold Crossing Events (TCE)** has lead to create a catalog of **planet candidates and false positives**, named **Kepler Objects of Interest (KOI)**. A KOI is a star suspected of hosting one or more transiting planets. KOIs come from a master list of 150'000 stars, which itself is generated from the Kepler Input Catalog (KIC). So, these stars are given the designation "KOI" followed by an integer number and for each set of periodic transit events associated with a particular KOI, a two-digit decimal is added to the KOI number. For exemple, KOI 718.01 is the first identifed TCE around the star KOI 718. The KOIs catalog, named **Kepler Objects of Interest (KOI) Activity Table** is available for use by the Astronomical community in selecting them for follow-up observations and further study.

At the beginning of the mission, KOIs lists were produced and vetted manually and relied on human judgment. As the Kepler mission matured, the shift in focus toward population-level studies pushed to develop automated and uniform way to produce these lists. Many projects has been developped to respond to this need, each with their own strategy and machine learning approach like Robovetter and Autovetter. The former used decision tree designed to mimic the manual process for rejecting false-positive TCEs, the latter using random forest model to classify TCEs based on features derived from Kepler pipeline statistics. Other project used unsupervised learning to cluster Kepler light curves with similar shapes and defined classification metrics using the distances between new light curves and TCEs with known labels. **Overall, the goal of these projects was to classify the observed TCEs in two categories: exoplanet candidate and false positive.**

Today, many of the KOI TCEs have been confirmed as exoplanet or false positive. Among the 9'564 TCEs recorded in the Cumulative Kepler Objects of Interest Activity Table, 2'303 TCEs are confirmed exoplanets, 4'841 TCEs are considered false positives and 2'420 TCEs remain candidates and need further investigations. The Kepler Space Telescope has provided tons of data, brigthness time-series, pipeline statistics, **fitted transit parameters**, **stellar properties** for each discovered TCEs. 

**In this capstone project, we propose to use machine learning techniques to identify, from the cumulative Kepler Objects of Interest Activity Table, the strongest TCE candidates. We will build models from the knowledge of the confirmed exoplanet and false positive TCEs, the TCE transit parameters and the hosting star stellar properties.**

## 1.2 Problem definition

The KOIs Activity Table references thousands of TCEs classified as `CONFIRMED` exoplanet and `FALSE POSITIVE` but also thousands of TCEs classified as `CANDIDATE`. These candidates would require more investigations to confirm if they are really exoplanets. 

In this project, we want to identify among the list of candidates, the candidates having the most chance to be exoplanets given the observed TCE transit parameters and the stellar properties of the hosting stars. To solve this problem, we propose to use machine learning techniques to train **binary classifier models**; models trained with only the confirmed exoplanet and false positive TCE observations. These models will then be used to reclassify the candidates that the classifiers will predict as exoplanets as strong candidates.

The list of TCEs and their corresponding classification labels (CONFIRMED|CANDIDATE|FALSE POSITIVE) are defined in the Cumulative Kepler Objects of Interest Table. From this table, we will extract two lists of TCEs:

1. The TCEs that will be used to train models; only the confirmed exoplanet and false positive TCEs
2. The candidate TCEs that will be assessed by the models 

We are interested to use high-level features that have been extracted from the stellar light curves and fitted during the Kepler processing pipeline or was already known before the Kepler mission. These features are all tabular data coming from the Kepler Data Validation Time-Series FITS files and explain:

* the stellar properties of the hosting star (radius, temperature, surface gravity, metallicity)
* the TCE transit parameters (transit period, transit duration, ingress duration, transit depth) 

We will train and tune **binary classifier models** with the confirmed exoplanet as the positive class and the precision metric to select the best model of each kinds. Finally, we will use our best models to asses the candidates to identify the strongest candidates, the ones that will be predicted as exoplanets by our classifiers.

### Disclaimer

Following discussions about the capstone proposal, it has been decided to reduce its scope, initially planned in two parts.  
The second part, using the transit models as features has been dropped and will be done as an aside personal project. 

Moreover, only the features being physical measures (the transit parameters and stellar properties) will be used to train the models. The data contains a lot of statistical features that could help but that are tied to the Kepler electronic and treatment pipeline. Some of these statistical features will be used to clean the data but not to train the models. It's an opiniated choice but we use tools to measure and then understand our world and I wish to use only these resulting measures to train the models. 

## 1.3 Abbreviations

* DV  - Data Validation Kepler pipeline module
* KIC - Kepler Input Catalog
* KOI - Kepler Object of Interest
* TCE - Threshold Crossing Events
* TPS - Transit Planet Search Kepler pipeline module

## 1.4 Links & Bibliography

Links:
* Cumulative KOIs Activity Table Online Viewer with CSV export feature    
https://exoplanetarchive.ipac.caltech.edu/cgi-bin/TblView/nph-tblView?app=ExoTbls&config=cumulative
* KOIs Activity Table Columns Definition:    
https://exoplanetarchive.ipac.caltech.edu/docs/API_kepcandidate_columns.html 
* Data Validatation Time-Series FITS files (all features are extracted from these files):  
https://archive.stsci.edu/missions/kepler/dv_files/tar_files_dr25/

This capstone project is inspired from the following publication:

* Identifying Exoplanets With Deep Learning: A Five Planet Resonant Chain Around Kepler-80 And An Eight Planet Around Kepler-90 https://www.cfa.harvard.edu/~avanderb/kepler90i.pdf

# 2. The data

## 2.1 Data preparation

In this project, the data are all publicly available and come from the following sources:

* The Cumulative Kepler Objects of Interest Activity Table 
* The Kepler Data Validation Time Series Fits Files 

In a nusthell, here is how these data will be used:

1. The KOIs Activity Table provides the list of KOIs and their classification labels:
    * CONFIRMED (exoplanet)
    * FALSE POSITIVE
    * CANDIDATE
    
    
The KOIs activity table provides also features explaining and justifying the false positive class.
    
2. The Data Validation Time Series FITS Files provide all the features:
    * the stellar properties
    * the fitted transit parameters 

These features must be extracted from 140 Go of FITS files.  


The `2_1_Data_Preparation.ipynb` notebook prepares the data and stores them in a SQLite database, ready for the cleaning and EDA steps.

## 2.2  Data overview

The dataframes and plots presented in this section come from the `2_2_Data_Overview.ipynb` notebook.

### 2.2.1 The classification target variable

The classification target variable is represented by the `koi_disposition` feature.  
There are three classes:

* Confirmed exoplanet TCEs (CONFIRMED)
* False positive TCEs (FALSE POSITIVE)
* Candidate TCEs (CANDIDATE)

The models will be trained with the false positive and confirmed exoplanets TCEs only. The candidates will be assessed by the best models of each types. The candidates predicted as exoplanet by the best classifier will be reclassify as strong candidate.

The following plots shows the labels distribution and the classes balance of the binaray classifiers we will train:

![Target variable distribution](./resources/images/2_2_1_target_variable.png "Target variable distribution")

About 20% of our data are candidates to assess and there are a majority of false positive TCEs. Concerning the training of our binary classifiers, we will have to deal with an **imbalance** of 60%/40% between the false positive (negative class) and the confirmed exoplanet TCEs (positive class) respectively.

There are many strategies to deal with classes imbalance like upsampling the minority class or downsampling the majorty class. In this project, we will instead **penalize the algoritms by applying weights** to each class according to their frequency.

### 2.2.2 The variables explaining the classification target variable

From the KOIs activity table, we have extracted variables explaining and justifying the classification target variable:

![Justification variables](./resources/images/2_2_2_justification_variables.png "Justification variables")

There are two classification variables:

1. The Exoplanet Archive Disposition variable (`koi_disposition`), our classification target variable.
2. The Disposition Using the Kepler Data variable (`koi_pdisposition`), the classification issued from the Kepler pipeline

The Exoplanet Archive Disposition variable is in fact the Disposition Using Kepler Data variable enriched with the known exoplanets coming from the Exoplanet Archive Confirmed Planet table. The Disposition Using Kepler Data variable has only two classes, CANDIDATE and FALSE POSITIVE, attributed according to the test results obtained during the Kepler pipeline. 

All the other variables describe the Disposition Using Kepler Data classification variable. They explain the attribution of the false positive class. Absence of comment and/or false positive flags mean that the observation can't be rejected as a false positive, so becomes a candidate. The score variable is the level of confidence of this classification, false positives must have scores near 0 and candidates scores near 1.

In this project, we profit the knowledge of the confirmed exoplanets and our goal is to use this knowledge to revisit the candidate list to identify the strongest ones among them. This wish and hope is only possible if we are able to train our model with the following data:

* A list of confirmed exoplanets
* A list of "confirmed" false positives

**These variables, except the classification target variable, will not be used to train the models because they would totally biased them**. Hovewer, they will be used extensively during the data cleaning and exploratory data analysis steps. We want to keep only the TCEs for which we have high confidence they are confirmed exoplanets or confirmed false positives. 

It's worth mentioning the meaning of the 4 false positive flags:

* **Not Transit-Like Flag**  (`koi_fpflag_nt`)  
A KOI whose light curve is not consistent with that of a transiting planet.  
This includes, but is not limited to, instrumental artifacts, non-eclipsing variable stars, and spurious (very low SNR) detections.

* **Stellar Eclipse Flag** (`koi_fpflag_ss`)  
A KOI that is observed to have a significant secondary event, transit shape, or out-of-eclipse variability, which indicates that the transit-like event is most likely caused by an eclipsing binary. However, self-luminous, hot Jupiters with a visible secondary eclipse will also have this flag set, but with a disposition of PC.

* **Centroid Offset Flag** (`koi_fpflag_co`)  
The source of the signal is from a nearby star, as inferred by measuring the centroid location of the image both in and out of transit, or by the strength of the transit signal in the target's outer (halo) pixels as compared to the transit signal from the pixels in the optimal (or core) aperture.

* **Ephemeris Match Indicates Contamination Flag** (`koi_fpflag_ec`)  
The KOI shares the same period and epoch as another object and is judged to be the result of flux contamination in the aperture or electronic crosstalk.

The `kepoi_name` variable identify uniquely each TCEs.

### 2.2.3 The features

The features have been extracted from the Kepler Data Validation FITS files.

They can be grouped in different themes:

* Stellar properties
* Stellar magnitudes
* Stellar location and motion
* Transit parameters
* Transit pipeline statistics

Here is a sample of the stellar properties: 

![Stellar properties](./resources/images/2_2_3_stellar_properties.png "Stellar properties")

Predicting the nature of a TCE by just looking at its hosting star is for sure a weak and naive approach. We dont expect these features to be strong predictors. Moreover, the Kepler mission was to monitor stars similar to our Sun, so our dataset contains probably no giant and dwarf stars. However, it's not impossible that the star population represented in our dataset present already some patterns that could improve our predictions by few percents.

These features explain in order the following star properties:

* the star identifier (`kepid`)
* the effective temperature
* the surface gravity
* the metallicity
* the star radius in solar radii
* the reddenining
* the extinction

The `numtces` variable indicates the number of TCEs registred around the star. The `quarters` variable is a bit string indicating the quarters during which the star has been monitored. The four and half year of mission duration is represented by 17 quarter bits. Stars can have missing quarters of data.


The transit parameters should be our strongest predictors. These high-level features have been extracted from the light curves by the transit planet search (TPS) and data validation (DV) Kepler pipeline modules. Here is a sample of these data: 

![Transit parameters](./resources/images/2_2_3_transit_parameters.png "Transit parameters")

where the features are in order:

* the transit period
* the transit depth
* the transit duration
* the ingress duration
* the impact
* the orbital inclination
* the ratio of planet distance to star radius 
* the ratio of planet radius to star radius
* the planet radius in earth radii

The magnitudes, location and motion features are expected to be weak predictors and will be studied during the EDA. The pipeline statistic features are out of the scope of this project but it's worth mentioning the signal-to-noise ratio and maximum multi-event statistics that will be used during the data cleaning and EDA steps.

# 3. The Exploratory Data Analysis

## 3.1 Data cleaning

The data and plots presented in this section come from the `3_1_EDA_Data_Cleaning.ipynb` notebook.

We will clean the data keeping only the TCEs for which we have high confidence they are confirmed exoplanets and confirmed false positives.

The main focus of this notebook is the analysis of the variables explaining the classification target variable:

* the Exoplanet Archive disposition (`koi_disposition`) VS the disposition using Kepler data (`koi_pdisposition`) classification variables
* the confidence score (`koi_score`)
* the disposition comment (`koi_comment`)
* the false positive flags (`koi_fpflag_nt`, `koi_fpflag_ss`, `koi_fpflag_co`, `koi_fpflag_ec`)

![Justification variables](./resources/images/3_1_justification_variables.png "Justification variables")

But, we will also :

* explore the quarters variable indicating the periods during which the stars have been monitored
* explore the transit signal-to-noise ratio and maximum multiple event statistic features because they are statistics related directly to the TCEs reliability
* handle the missing values

Note that the feature selection has already been done in the `2_1_Data_Preparation.ipynb` notebook

### 3.1.1 The Exoplanet Archive disposition VS the disposition using Kepler data

The Exoplanet Archive Disposition variable is in fact the Disposition Using Kepler Data variable enriched with the known exoplanets coming from the Exoplanet Archive Confirmed Planet table. We want to drop any inconsistency between these two classification variables, with the assumption that confirmed exoplanets should have been classified as candidates by the Kepler pipeline. 

Here is the counts of observation for each Disposition Using Kepler Data, grouped by Exoplanet Archive Disposition :

![](./resources/images/3_1_1_classification_variables_inconsistencies.png "")

The 7 mismatches between the two classification variables have been dropped.

### 3.1.2 The confidence score

The score is the confidence in the Disposition Using Kepler Data defined as:

>A value between 0 and 1 that indicates the confidence in the KOI disposition. For CANDIDATEs, a higher value indicates more confidence in its disposition, while for FALSE POSITIVEs, a higher value indicates less confidence in that disposition. The value is calculated from a Monte Carlo technique such that the score's value is equivalent to the fraction of iterations where the Robovetter yields a disposition of CANDIDATE.

We want to train models with TCEs classified with high score of confidence. The analysis of the score distribution per disposition will help us to select these observations. Because this variable is strongly skewed,  plotting cumulative histrograms for the candidates and exoplanets and an inverse cumulative histrogram for the false positives, with a logarithmic scale, will provide better insight. 

![](./resources/images/3_1_2_score_distribution.png "")

Most false positives have confidence scores near 0 and exoplanets near 1 but there are TCEs with low confidence scores to drop. The candidates have more spreaded scores but overall, they are concentrated with a score between 0.8 and 1, indicated by the "quadratic" trend.

We make the decisions to drop :

* the 182 false positive TCEs with a score greater than the 95% quantile (= 0.252)
* the 410 candidate TCEs with a score below 0.8
* the 115 confirmed exoplanet TCEs with score lower than the 5% quantile (= 0.839)

707 observations have been dropped in the process.

### 3.1.3 The comment

The comment variable is the description of the reason why an object's disposition has been given as false positive. It may also contain a list of the minor tags as set by the Robovetter. These tags are described here: https://exoplanetarchive.ipac.caltech.edu/docs/koi_comment_flags.html

We want to drop the observations having rare tags.  
The tag distributions per disposition will allow to do identify the tags with sample size statistically not significant:

#### Tags attributed to the confirmed exoplanets

![](./resources/images/3_1_3_exoplanet_tags.png "")

We keep only the exoplanets having the NO_COMMENT and the CENT_KIC_POS tags, all others having a sample size statistically not significant.  
The CENT_SATURATED tag has also been dropped because the documentation states that this tag is a warning that the TCE can not be reliably evaluated. 

83 confirmed exoplanets have been dropped.


#### Tags attributed to the false positives

![](./resources/images/3_1_3_false_positive_tags.png "")

The comment variable describes essentially the false positive class.  
We keep only the false positive TCEs having tags with a sample size greater than 34 observations, the square root of the most frequent tag count.

189 false positives have been dropped.


#### Tags attributed to the candidates

![](./resources/images/3_1_3_candidate_tags.png "")

We decided to keep the candidate TCEs with the same tags than the confirmed exoplanets: NO_COMMENT and CENT_KIC_POS. Moreover, the documentation about the CENT_FEW_DIFFS, CENT_FEW_MEAS and CENT_SATURATED tags encourage to drop them even if their sample sizes are statistically significant.

154 candidates have been dropped.


During this analysis, 426 observations have been dropped.

### 3.1.4 The false positive flags

The disposition using Kepler data classification variable is set in function of the 4 following flags:

* Not Transit-Like Flag (`koi_fpflag_nt`)
* Stellar Eclipse Flag (`koi_fpflag_ss`)
* Centroid Offset Flag (`koi_fpflag_co`)
* Ephemeris Match Indicates Contamination Flag (`koi_fpflag_ec`)

Exoplanets and candidates should have all four flags set to 0 and the false positives should have at least one of these flags set to 1.  
Only one observation have been dropped, an exoplanet with the "not transit-like flag".

More interesting is the analysis of the false positive subcategories given by their flags.  
To lead this analysis, we have engineered a false positive flag bitstring variable and counted the number of false positives per bitstring :

![](./resources/images/3_1_4_false_positive_flags.png "")

The most significant bit is the not transit-like flag followed in order by the stellar eclipse flag, the centroid offset flag and the ephemeris match flag.

The most represented category of false positives, 33% of them, are stellar eclipses, binary stars most likely.  
The three other pure categories, identified by only one flag are in order:
* the centroid offset flag (0010) representing 13% of the false positives
* the not transit-like flag (1000) representing 10% of the false positives
* the ephemeris match indicates contamination flag (0001) representing only 3% of the false positives

During the exploratory data analysis, it will be interesting to investigate about these subcategories and check if they obey to different patterns.  
Note that 41% of the false positives have been rejected for more than one reason.

### 3.1.5 The quarters of measurement

The monitoring of the stars is splitted in 17 measurement quarters. Between each quarter, the Kepler Telescope needed to be reoriented to receive energy from its solar panels. For some reasons, some stars has not been monitored during the 17 quarters, resulting of a lot of missing data in their light curves from which the transit models are derived. The nomenclature says that the Kepler pipeline handles these missing data with some interpolation. 

To build our models, we will get rid of the observations with missing measurement quarters or more precisely keep only the observations having contiguous quarters of measurement. Missing quarters at the beginning or end of the Kepler lifespan are less a concern.

The quarters variable (`quarters`) is a bit string with the first quarter indicated by the most significant bit, and so on...  

For this analysis, we have engineer two variables:

* A variable indicating the number of quarters (number of bits at 1)
* A variable indicating if there are missing quarters between measurement quarters (bit at 0 between bits at 1)

This second variable has allowed to drop 1'478 TCEs with missing quarters of data, TCEs less reliable by definition.

More interesting is the analysis of the distribution of TCEs per number of contiguous quarters of measurement, stacked by disposition:

![Quarters of measurements](./resources/images/3_1_5_quarters.png "Quarters of measurements")

Fortunately, most of the TCEs have been recorded around stars monitored during the 17 quarters. From the sample sizes, it makes sense to keep only the observations with at least 14 measurement quarters.

As we can see, the TCEs with few measurement quarters are mostly false positives. It can probably be explained by the fact that the probability to detect an exoplanet during a so short period of time is low. The full four and half year corresponds to only four Earth transits. A lot of chance would be required to detect a planet like Neptune with its period of revolution of 165 years. The analysis of the transit period and duration will be done later but we can expect to have TCEs with short transit periods.

By keeping only the TCEs with at least 14 quarters, we have dropped 172 TCEs more pushing the total to 1'650 observations.

### 3.1.6 The maximum multiple event statistic and signal-to-noise ratio 

The pipeline statistics are out of the scope of this project but the maximum multiple event statistic (`maxMES`) and signal-to-noise ratio (`snr`) variables are direct measure fo the reliability of the observed TCEs and their analysis proved to be interesting during the project proposal.

#### The maximum multiple event statistic

According to the documentation, the maximum multiple event statistic can be exploited to help filter and sort samples of TCEs for the purposes of discerning the event quality, determining the likelihood of planet candidacy, or assessing the risks of observational follow-up. The MES estimates the signal to noise ratio of the putative transit-like sequence against the measurement noise. 

![](./resources/images/3_1_6_maxMES_distribution.png "")

Most of the TCEs tend to have low max MES. In astronomy, signals are often faint. The second "bumb" appears to be false positives, probably the stellar eclipses; their higher max MES, meaning they are easier to detect in the light curves :

![](./resources/images/3_1_6_maxMES_false_positives.png "")

The stellar eclipse false positives are efffectively the second bumb in the histrogram.

#### The transit signal-to-noise ratio

This variable is defined in the documentation as the transit depth normalized by the mean uncertainty in the flux during the transits.

![](./resources/images/3_1_6_SNR_distribution.png "")

The SNR distributions are clearly similar to the maxMES distributions, indicating evident correlation.  
Note that 8 TCEs with negative log transformed SNR have been dropped.


#### Correlation of the signal-to-noise ratio and the maximum multiple event statistic

![](./resources/images/3_1_6_SNR_vs_maxMES.png "")

The log transformed two variables are clearly linearly correlated. However, the exoplanets appears to be less spreaded and better aligned. The candidates tends to follow the exoplanet trend even if most of them have low SNR and maxMES values, a region where it will be more challenging for our models to classify correctly the TCEs. The candidates with a maxMES between 1.5 and 2.5 are probably the most promising. 

The right scatter plot shows that the false positives subcategories form clusters. It will be probably easy for our models to classify correctly the stellar eclipses but more challenging to classify the other kinds of false positives. We have to be aware about the features that could help to identify them specifically.

We make the choice to don't use the statistics to train our models but in other project, we could build a linear model with only the confirmed exoplanets observations and engineer a variable representing the Euclidean distance of each data point to this model. Such a variable could say that observations far from the linear model have low probability to be an exoplanet, but not the reciproque condition.

This same logic can be used to drop some outliers, TCEs too far from the linear trend. The training of simple linear model and the analysis of the distance of each data points from the model as allow to drop 14 TCEs.

### 3.1.7 Handling the missing values 

![](./resources/images/3_1_7_missing_values.png "")

These missing values have been handled as follows :

- TCEs with missing `teff`, `logg`, `feh`, `radius` values have been dropped (important stellar properties)
- TCEs with missing `CCDP` statistics have been dropped (reliability statistic)
- Missing proper motion `pmdec`, `pmra`, `pmtot` values have been replaced by 0 (most frequent value, > 68%)
- Extinction and reddening, `ebminusv` and `av` missing values have been replaced by the values of the stars having the nearest temperature
- Band magnitudes missing values have been replaced by the sum of the kepler magnitude and the median delta between the respective band magnitude and the kepler magnitude
- Color missing values have been simply computed according to their definition formula

During this process, we have discovered that the `ebminusv` and `av` features are colinear.  
Moreover, the `grcolor` and `gkcolor` are engineered variables, respectively `gmag - rmag` and `gmag - kmag`.

## 3.2 EDA of the stellar properties

The data and plots presented in this section come from the `3_2_EDA_features.ipynb` notebook.

We will explore the stellar properties even if we expect these features to be weak predictors. Indeed, it sounds naive at best to think that it's possible to predict the presence of an exoplanet only by knowing their hosting star temperature, radius and few more properties. However, we can also expect that dwarf and giant stars have lower probability to host exoplanets. Because the Kepler mission was to monitor stars similar to the Sun, there are probably no giant and dwarf stars in our dataset but it doesn't mean that we can't discover some trends among the biased population it represents. We could improve the accuracy of our models few percents, maybe...

The goals of this EDA are :

* Compute the features skewness and kurtosis and engineer transformed variable when necessary 
* Plot their histograms and distributions splitted by label and false positive subcategories
* Engineer threshold indicator variables when the distributions show some trends
* Discover potential correlations with pairplots and multivariate scatter plots
* Drop potential outliers


Here are the stellar properties:
![](./resources/images/3_2_stellar_properties.png "")

### 3.2.1 Skewness and kurtosis

First, we want to check if they follow normal distribution by computing the skewness and kurtosis of their distribution. We will also compute the skewness and kurtosis of their respective logarithmic and square root transformed variables :

![](./resources/images/3_2_1_skewness_kurtosis.png "")

To have variables with distributions as similar as possible to normal distributions, we have to:

* engineer an effective temperature, square root transformed variable (`teff_sqrt`)
* engineer a stellar radius, log10 transformed variable (`radius_log10`)

### 3.2.2 Effective temperature analysis

![](./resources/images/3_2_2_effective_temperature.png "")

Exoplanets tends to orbit around stars with lower temperatures.  
We can engineer a threshold indicator variable represented by the green dashed line, following the rule :

> 1 when \\(\sqrt{temperature} \geq 83\\) (false positive indicator)  
> 0 else

### 3.2.3 Surface gravity

![](./resources/images/3_2_3_surface_gravity.png "")

Stars with low surface gravity tend to be the host of false positives only.  
We can engineer a threshold indicator variable represented by the green dashed line, following the rule :

> 1 when \\(\log_{10}(gravity) \leq 3.85\\) (false positive indicator)  
> 0 else

### 3.2.4 Stellar radius

![](./resources/images/3_2_4_stellar_radius.png "")

The stellar radius variable clearly demonstrates the bias of our dataset. Most of the stars have size similar to the Sun. The probability to find exoplanets around stars 3 times bigger than the Sun is low.

We can engineer a threshold indicator variable represented by the green dashed line, following the rule :

> 1 when \\(\log_{10}(radius) \geq 0.36\\) (false positive indicator)  
> 0 else

The stellar radius distributions look similar to the surface gravity distributions but inverted.  
These two features are probably correlated, it would make sense that bigger stars have lower surface gravity :

![](./resources/images/3_2_4_radius_gravity_correlation.png "")

As expected, the two features are inversely proportional. On the right plot, we can see that the surface gravity is nearly linearly proportional to the inverse of the stellar radius. Surprisingly, I would expect the surface gravity linearly proportional to the inverse of the square of the radius... I don't explain it...

The two variables are nearly colinear and keeping both will probably not help our models.  
Spoiler: we will only keep the stellar radius and its associated threshold

Note that 7 outlier TCEs hosted by star having particularly small radius have been dropped.

### 3.2.5 Reddening and extinction

We already know that the reddening and extinction are colinear variables. We will only keep the reddening feature to train our models.

![](./resources/images/3_2_5_reddening.png "")

Reddening tends to be lower for stars hosting exoplanets. It's maybe because the nebula from which the stars are born have also formed planets. The resulting dust cloud being less dense, the light is also less scattered...

We can engineer a threshold indicator variable represented by the green dashed line, following the rule :

> 1 when \\(reddening \geq 0.185\\) (false positive indicator)  
> 0 else

### 3.2.6 Metallicity

![](./resources/images/3_2_6_metallicity.png "")

Stars hosting exoplanets tends to have higher metallicity, particularly compared to stars hostings stellar eclipse false positive TCEs.

We can engineer a threshold indicator variable represented by the green dashed line, following the rule :

> 1 when \\(metallicity \leq -0.57\\) (false positive indicator)  
> 0 else

### 3.2.7 Pairplots and multivariate analysis

Now, we know more about the variables distributions, it's time to investigate about their relationships.  
Pairplots are an efficient way to visualize and explain them.

![](./resources/images/3_2_7_paiplots.png "")

The following relationships looks particularly interesting and could show some clustering patterns :

* temperature VS radius
* metallicity VS radius
* reddening VS temperature

![](./resources/images/3_2_7_multivariate_analysis.png "")

The gray dashed lines are the threshold previously defined. The exoplanets form a less compact cluster but unfortunatly it always overlap the more spreaded false positives cluster.

We can maybe explain the following trends :

* left plot: bigger stars have lower effective temperature because their surface grows as the square of their radius. However, their producted energy don't grow as quickly in function of their radius. The density of the star could probably explain the split at (0.2, 80)

* right plot: reddening and effective temperature are positively correlated because higher is the star temperature, shorter is the main wavelength emitted. Short wavelengths are more scattered than high wavelengths, so the positive correlation. 

### 3.2.8 Summary

With no surprise, the stellar properties appears to be weak predictors. We have engineered some threshold indicator variables able to easely identify some false positives, TCE observations looking as outliers in our dataset but that aren't.

## 3.3 EDA of the transit parameters

In this section, we will explore the transit parameters as we did for the stellar properties. We expect the transit parameters to be strong predictors because they are features extracted from the light curves provided by the Kepler Space Telescope.

The high-level goal of this EDA is to find predictors able to identify the false positives and if possible to isolate the subcategories/clusters of false positives.

We will :
* compute the features skewness and kurtosis and engineer transformed variable when necessary
* plot their histograms and distributions splitted by label and false positive subcategories
* engineer threshold indicator variables when the distributions show some trends
* Discover potential correlations with pairplots and multivariate scatter plots

Here are the transit parameters:
![](./resources/images/3_3_transit_parameters.png "")

### 3.3.1 Feature engineering

Given the meaning of the transit parameter features, we have engineered the two following ratio variables :

1. The ratio of the ingress duration to the transit duration
2. The ratio of the transit duration to the transit period

Our intuition says that these two ratio should bring some information about the "planet" radius and distance from its hosting star. The ratio of the ingress duration to the transit duration should be proportional to the ratio of the "planet" radius to star radius. The ratio of the transit duration to the transit period should be proportional to the ratio of the star radius to the planet distance.

We will start the EDA by the analysis of these two new features...

### 3.3.2 Skewness and kurtosis analysis

First, we have to check if the features follow normal distribution by computing the skewness and kurtosis of their distribution. We will also compute the skewness and kurtosis of their respective logarithmic and square root transformed variables :

![](./resources/images/3_3_2_skewness_kurtosis.png "")

To have variables with distributions as similar as possible to normal distributions, we should engineer the following transformed variables:

* \\(\log_{10}(tperiod)\\) (`tperiod_log10`)
* \\(\log_{10}(tdepth)\\) (`tdepth_log10`)
* \\(\log_{10}(tdur)\\) (`tdur_log10`)
* \\(\log_{10}(indur)\\) (`indur_log10`)
* \\(\log_{10}(indur\_tdur\_ratio)\\) (`indur_tdur_ratio_log10`)
* \\(\log_{10}(tdur\_tperiod\_ratio)\\) (`tdur_tperiod_ratio_log10`)

### 3.3.3 Ratio of the ingress duration to the transit duration

![](./resources/images/3_3_3_indur_tdur_ratio.png "")

The stellar eclipse false positives have significantly higher ratio of ingress duration to transit duration ratio. 

By engineering the threshold indicator variable represented by the green dashed line, we can easely identify most of them:

> 1 when \\(log_{10}(indur\_tdur\_ratio) \geq -0.38\\)  
> 0 else

Here are some statistics about the number of TCEs identified by this variable:

![](./resources/images/3_3_3_indur_tdur_ratio_stats.png "")


Note that they are some candidates above this threshold meaning they are probably not strong candidates...

### 3.3.4 Ratio of the transit duration to the transit period

![](./resources/images/3_3_4_tdur_tperiod_ratio.png "")

The ephemeris match false positives have significantly higher ratio. The not transit-like false positive distribution is particularly interesting. They are well spreaded with a mean significantly bellow the exoplanet mean but with 50% of them appearing as extreme values in regards of the exoplanet range.

By engineering the two threshold indicator variables represented by the green dashed lines, we can easely identify more than 25% of the ephemeris match and 50% of the not transit-like false positives:

> 1 when \\(log_{10}(tdur\_tperiod\_ratio) \geq 0.4\\)  
> 0 else

> 1 when \\(log_{10}(tdur\_tperiod\_ratio) \leq -1.5\\)  
> 0 else

Here are some statistics about the number of TCEs identified by this variable:

![](./resources/images/3_3_4_tdur_tperiod_ratio_stats.png "")

### 3.3.5 Transit depth

![](./resources/images/3_3_5_tdepth.png "")

The transit depth appears to be also a strong predictor. The higher transit depth of the stellar eclipse false positives can probably be explained by the fact that stars beeing bigger than planets, the eclipses are more often total / less partial, resulting to a deeper brightness dimming.

By engineering the two threshold indicator variables represented by the green dashed lines, we can easely identify more than 50% of the stellar eclipse and about 25% of the ephemeris match :

> 1 when \\(log_{10}(tdepth) \geq 4.4\\)  
> 0 else

> 1 when \\(log_{10}(tdepth) \leq 1.45\\)  
> 0 else

Here are some statistics about the number of TCEs identified by this variable:

![](./resources/images/3_3_5_tdepth_stats.png "")


A study of the TCEs identified by the threshold indicator variables engineered until now shows that these two new variables are valuable allowing to identify new false positives :

![](./resources/images/3_3_5_scatter_plots.png "")

The engineered threshold indicator variables do a good job to identify the false positives, especially the stellar eclipse ones.

A 3D scatter plot give good insight about the topologies of the clusters: 

![](./resources/images/3_3_5_3dplot.png "")

Without knowning the project, I would guess this plot related to the Lac Leman/Jura region... :)

* The stellar eclispses form the "wall" cluster
* On the right plot, the left cluster are mostly ephemeris match and not transit-like false positives
* the rigth cluster are mostly not transit-like false positives

Until now, we have not been able to identify the centroid offset subcategories of false positives...

### 3.3.6 Location and next steps

We could procede the same for all other features but the new threshold variables would suffer some diminushing returns. The following features appeared to be weak and uninteresting predictors, so their analysis are not presented but have been checked in an aside notebook:

* all the magnitudes variables, presenting no trend at all
* the motion features, most of the TCEs have no motion

However, plotting a map of the location of each TCE, their declination VS their right ascension proved to be interesting:

![](./resources/images/3_3_7_location.png "")

There are 17 regions as the number of the quarters and reorientation of the Kepler Space Telescope. We can see that the regions at the lowest declination and highest right ascension tends to have more false positives.

I think it could be the consequence of a higher density of stars in these regions, pointing towards the center of the galaxy that could increase the detection of centroid offset and stellar eclipse false positives.

Instead of keeping the `ra_obj` and `dec_obj` features to train our models, we have engineered a categorical variable attributing to each TCEs the cluster ID they belongs to. This classification has been done by training a
KMeans unsupervised learning classification model.

Note that the clusters have been ordered in a manner that the rows have incremental numbers from top to bottom and columns from left to right, meaning that the clusters at the highest right ascencion and lowest declination have the highest cluster identifiers.

The following two bar plots shows that clusters with an ID greater than 42 have significantly more false positive  and clusters with an ID below 23 less. Moreover, the analysis of the number of false positives per clusters, splitted by their subcategories shows that the clusters with the highest rigth ascension and lowest declination (clusters > 43) contains significantly more centroid offset false positives.

![](./resources/images/3_3_7_barplot_disposition.png "")
![](./resources/images/3_3_7_barplot_fp.png "")

The engineering of an indicator variable flagging the TCEs located in the region with the most centroid offset false positives (clusters 44, 45, 52, 54) will prove to be more interesting than the cluster identifier ordinal categorical variable.

### 3.3.7 Summary

As expected, we found strong predictors among the transit parameters features. Most of them must be log transformed to have near normal distribution. By the engineering of many threshold indicator variables, we have been able to define already good boundaries between the different false positive clusters and the confirmed exoplanets. Thank to the ratio of ingress duration to transit duration and ratio of transit duration to transit period engineered features better than the individual features from which they are computed. 

The overall EDA has allowed us to gain insight about the data, clean them and engineer many interesting features.  
We can now prepare the final datasets that will be used to train and test the models.

# 4. Machine Learning

## 4.1 Machine Learning Plan

The data and plots presented in this section come from the `4_1_Model_Datasets.ipynb` notebook. 

### 4.1.1 Dataset preparation

#### Features selection

Here is the list of features that will be used to train the models as named in the database :

* `feh`: metallicity
* `ebminusv`: reddening 
* `teff_sqrt`: effective temperature (sqrt)
* `radius_log10`: stellar radius (log10)  
* `teff_maxthresh`: effective temperature max threshold indicator variable
* `radius_maxthresh`: stellar radius max threshold indicator variable 
* `ebminusv_maxthresh`: reddening max threshold indicator variable
* `feh_minthresh`: metallicity min threshold indicator variable
* `tperiod_log10`: transit period (log10)
* `tdepth_log10`: transit depth (log10)
* `tdur_log10`: transit duration (log10)
* `indur_log10`: ingress duration (log10) 
* `indur_tdur_ratio_log10`: ratio of the ingress duration to the transit duration (log10) 
* `tdur_tperiod_ratio_log10`: ratio of the transit duration to the transit period (log10)
* `impact`: impact
* `indur_tdur_ratio_maxthresh`: ratio of the ingress duration to the transit duration, max threshold indicator variable 
* `tdur_tperiod_ratio_minthres`: ratio of the transit duration to the transit period, min threshold indicator variable
* `tdur_tperiod_ratio_maxthresh`: ratio of the transit duration to the transit period, max threshold indicator variable
* `tdepth_minthres`: transit depth min threshold indicator variable
* `tdepth_maxthresh`: transit depth max threshold indicator variable
* `location_cluster_fpco`: location cluster > 43 indicator variable

There are 21 features.  
The colinear and near colinear variables have been excluded : surface gravity `logg` and extinction `av`.

#### Target variable distribution

After the EDA, it remains 4'875 observations :

![](./resources/images/4_1_target_variable_counts.png "")

#### Candidates dataset

The 977 candidates will be assessed by the best models.  
They are not used to train, tune and test the models and grouped in their own dataset.

#### Split of the training and test sets

There are 3'898 false positive and exoplanet TCEs to split to create a training set and a test set.

The split is done as is:

* Stratified split to have the same classes distribution in each set
* 3'000 observations for the training set, 898 observations for the test set (~ 75%/25%)

![](./resources/images/4_1_training_set_counts.png "")

![](./resources/images/4_1_test_set_counts.png "")


All the prepared datasets have been saved in a NPZ file and ready to be used without any transformation to train, tune and test the models.

### 4.1.2 The Metric

#### Some basic theory

The goal of a binary classifier is to predict the class of new data point among two classes. Let's call these two classes and outcomes: Positive (P) and Negative (N).

To evaluate this kind of models, we compare the predicted class to the actual class. There are four possible outcomes:

1. True Positive (TP): The actual class is positive and the predicted class is also positive
2. True Negative (TN): The actual class is negative and the predicted class is also negative
3. False Positive (FP): The actual class is negative but the predicted class is positive 
4. False Negative (FN): The actual class is positive but the predicted class is negative

To avoid confusion between the  `koi_disposition` FALSE POSITIVE label and this False Positive (FP) outcome, we will only use the outcomes abbreviations. The term false positive will always refer to the label.

To evaluate a model, we can count these TP, TN, FP, FN values and present them in a matrix called confusion, contingency or classification matrix. 

The following metrics can be computed :

**Accuracy**  
The accuracy is the fraction of predictions our model got right.  
We can deduct the fraction of misclassifications but they are supposed to be 50% FP / 50% FN.
 
$$accuracy=\frac{TP+TN}{TP+TN+FP+FN}$$

The accuracy metric is often not really interesting. FP and FN have often deep meaning and consequences according to the business domain and we prefer metrics taking them into account.

**Precision**  
The precision is the ability of the classifier not to label as positive a sample that is negative. 

$$precision=\frac{TP}{TP+FP}$$

A high precision means that when our model predict a positive (P) outcome, there is high chance that the true outcome is also positive. However, it's important to note that the precision metric doesn't take into account the FNs. A model can have a high precision but also misclassify/miss half the actual positive (P) cases.

**Recall**  
The recall is intuitively the ability of the classifier to find all the positive samples

$$recall=\frac{TP}{TP+FN}$$

A high recall means that the model is a good positive cases (P) detector. However, this metric doesn't take into account the FPs. A model could be a very good positive cases (P) detector but also wrongly consider actual negative cases (N) as positive (P).


It appears that there is a trade-off between the precision and recall metrics. By wanting to miss no positive cases (recall optimization), the model must be less selective and so will predict more FPs and its precision will decrease. Reciprocally, optimizing the precision to be sure that when we predict a positive case, it is really a positive case, requires a more selective model that will increase the number of FN predictions, so decrease its recall.

The decision to optimize one metric or the other is driven by business/domain considerations :

* Constrained resources (budget, time, trust): Select the precision metric and accept the resulting recall (lack of performance)
* Performance target: Select the recall metric and accept the resulting precision (budget increase)

There exist other metrics derivable from the TP, TN, FP, FN counts, we will also compute one of them : 

**F1 Score**

The F1 score can be interpreted as a weighted average of the precision and recall, where an F1 score reaches its best value at 1 and worst score at 0.  
The relative contribution of precision and recall to the F1 score are equal.

$$f1=\frac{2 * precision * recall}{precision + recall}$$

#### The strongest exoplanet candidates, a precision scenario

In this project, we want to identify among the candidates, the candidates having the more chance to be exoplanets. Most of the candidates in the list are probably already good to very good candidates.

We are not interested to identify all of them but only the strongest one, the candidates that have all the properties to be for sure an exoplanet, or at least should be. It's a typical time/budget constraint scenario were only the most promising cases can be considered and in this situation, the precision metric is the good choice.

So, **we will tune our models to optimize the precision metric**.  

We will compute the following metric to have a better understanding of our models:

* Precision
* Recall
* Accuracy
* F1 Score

At equivalent precision score, model with the best recall will be considered better.

### 4.1.3 The models

We will train the following supervised binary classifier models:

* A **decision tree** for simplicity and a more advanced baseline model. The robovetter project was based on decision trees. Decision tree unfortunately overfit quickly the data.

* A **random forest** model because given the EDA, we can expect good results. Moreover, random forest are better than decision tree because they reduce the variance without increasing the bias. The Autovetter project was based on random forest models.

* A **k Nearest Neighbors** model because kNN models are very intuitive, don't have to learn some coefficients and make no assumption about the linearity or non-linearity of the data.

* A **logistic regression** model that is a probabilistic classifier by definition. The probability logits are related to a linear function in the feature space. We expect this model to be weaker.

* **Support vector machine models with linear and RBF kernel**. SVM models are often used to solve classification problems. A linear support vector machine model try to find an optimal hyperplane with margins to separate two classes. From the EDA, we know  that it is probably not possible to find such an hyperplane. That is why we will also build a SVM model with a RBF kernel. We expect good results from this last model.

### 4.1.4 Methodology

In a nutshell, here is the methodology we will follow to train, tune and test all the models :

1. Stratified split of the dataset: ~75% for the training + validation set, ~25% for the test set
2. Hyperparameter tuning by grid search and 5-fold cross-validation 
3. Selection of the best model/set of hyperparameters in regards to the precision metric
4. Best model training of the full training set 
5. Best model evaluation on the test set

Finally, the best models of each kind will be used to assess the candidates.

### 4.1.5 The Pipeline

Here is the typical pipeline that will be used to train and tune the models :

![](./resources/images/4_1_pipeline.png "")

The features are standardized because most models works better when its the case:

* the scales have influence on the decision boundaries of SVM models
* stochastic gradient descent algortihms prefers standardized data

Because half of our features are not strong predictors, we can also expect that reducing the number of dimensions with principal components analysis (PCA) will improve the predictions of our models.

### 4.1.6 PCA Analysis

Here is a PCA scree plot :

![](./resources/images/4_1_scree_plot.png "")

![](./resources/images/4_1_variance_explained.png "")

We will tune the PCA number of components by grid search from 80% to 100%, according to the capstones above.

### 4.1.7 Baseline model

We know that the training and test set contains 58.7% of false positives and 41.3% of exoplanets, meaning that the baseline precision score if we would always predict an exoplanet outcome is 41.3%.

However, during the EDA, we have been able to engineer threshold indicator variables identifying easely a lot of false positives.  
It would be more realistic to compare our models to a baseline without all these false positives.

Here is the remaining proportions of false positives and exoplanets: 
![](./resources/images/4_1_baseline_model.png "")

Our baseline model has a precision of 66.4%.  
To be better than the EDA alone, the models must have a precision score above 66.4%.

We can now follow our plan and train our models !

## 4.2 Decision Tree

The data and plots presented in this section come from the `4_2_Decision_Tree.ipynb` notebook.

Decision trees tend to quickly overfit the training data as its depth increase. Because trees segment the predictor space into a number of regions, we can expect that applying PCA to reduce the number of dimensions of this space by building higher level predictors keeping a huge proportion of the original variance should be profitable.

The following hyperparameters have been tuned by grid search and 5-folds cross-validation:

* PCA number of components 
* Maximum depth of the tree 
* Number of features to consider when looking for the best split

Note that because the imbalance of our target variable, we apply weights inversely proportional to class frequencies: `class_weight='balanced'`.

500 models have been trained, here are the results presented as three scatter plots:

![](./resources/images/4_2_decision_tree_tuning.png "")

The overfitting induced by the increase of the maximum tree depth is clearly visible. As soon as the tree depth is greater than 6, the difference between the validation and training precision scores are greater than 5%.

Most of the models have a standard deviation between 1% and 4%. It's okish but a rule of thumb say that it's generally not good enough for production. Good models should have validation standard deviation less than 1%.

Given the results, our wish would be to find a model satisfying the following conditions:

* validation standard deviation <= 1%
* mean validation precision >= 75%
* delta mean precision <= 5%

Only one model fullfill all these conditions :

![](./resources/images/4_2_decision_tree_best_models.png "")

We can plot the precision training and validation curves in function of the tree depth :

![](./resources/images/4_2_decision_tree_curves.png "")

The selected model has the best validation standard deviation but is also the first maxima.

By training the model on the full training set and evaluating it on the test set, we obtain the following results:

![](./resources/images/4_2_decision_tree_results.png "")

The training and test precision scores are close, even equal indicating that our model generalizes well the data.  
This model is clearly better than the baseline models. With a precision score of 82% and a recall of 86%, it appears to be a well balanced model.

## 4.3 Random Forest

The data and plots presented in this section come from the `4_3_Random_Forest.ipynb` notebook.

Decision trees tend to have low bias but high variance, the cause of their apparant lack of robustness. Random forest reduces this variance. It achieves it by bootstraping the training data and building one tree from each bootstraped dataset. Each tree will predict a class for a test data point and the most-frequent predicted class will be the one given to the point. As a result, random forest reduces the variance without bias trade-off but at the expense of the model interpretability. This approach is called bagging. Moreover, each tree split can be done only on a randomly drawn subset of the predictors. As a result, random forest build a forest of heterogeneous/decorralated trees; the strongest predictors are not always at the top of the trees.

The following hyperparameters have been tuned by grid search and 5-folds cross-validation:

* PCA number of components 
* Number of trees
* Maximum depth of the trees
* Number of features to consider when looking for the best split

Note that because the imbalance of our target variable, we apply weights inversely proportional to class frequencies: `class_weight='balanced'`.


1050 models have been trained, here are the results presented as three scatter plots:

![](./resources/images/4_3_random_forest_tuning.png "")

The overfitting induced by the increase of the maximum tree depth is clearly visible and form clusters. It appears that the best models should have a maximum depth of at least 10 and that above a depth of 20, we can't really expect better precision score but more overfitting.

Most of the models have a standard deviation between 1% and 3%. It's okish but a rule of thumb say that it's generally not good enough for production. Good models should have validation standard deviation less than 1%.

Given the results, our wish would be to find a model satisfying the following conditions:

* validation standard deviation <= 1%
* mean validation precision >= 85%

![](./resources/images/4_3_random_forest_best_models.png "")

Except the second model that bring no advantage over the first one, each others could be selected with good reason:

* the first one has the lowest validation standard deviation, 87% validation score, 98% training score
* the third one has a lower 86% validation score, but less overfitting because its 92% training score
* the last one is in between with a slighlty better 87% validation score and 95% training score

We will choose the first model as the best one.  
I tend to think that choosing the lowest validation strandard deviation is the way to go as soon as the mean validation score is near the best score.

By training the model on the full training set and evaluating it on the test set, we obtain the following results:

![](./resources/images/4_3_random_forest_results.png "")

The training and test precision scores are closer and better than expected by cross-validation. The overfitting remains acceptable given the nature of the model.
  
This model is clearly better than the baseline models and an improvement over our best decision tree model.  
With a precision score of 89% and a recall of 84%, it appears to be a well balanced model.

## 4.4 K Nearest Neighbors

The data and plots presented in this section come from the `4_4_kNN.ipynb` notebook.

K Nearest Neighbors models are very intuitive and great in the sense that they don't try to learn some coefficients and don't make any assumption about the linearity or non-linearity of the data. Data points are simply points in an N dimensional space with a target value. Features are only used to compute the distance between data points. The predicted target value of a test data point is attributed in function of its k nearest neighbors data point target values. So, there are in fact two functions behind the scene: 

* One function that computes the distance (L1, L2, or something else if we want)
* One function that returns a predicted target value from the class of the k nearest data points (the class with the most occurrence for example)

The following hyperparameters have been tuned by grid search and 5-folds cross-validation:

* PCA number of components
* Number of nearest neighbors
* Function computing the distance: L1 and L2
* Weight function: uniform and distance

Note that the knn model doesn't support the class_weight parameter to manage the imbalance of our target variable. Aside of this notebook, I tried to downsampling the majority class but it wasn't worth the effort. Dropping 500 false positives has more negative effect than the classes imbalance.


620 models have been trained, here are the results presented as three scatter plots:

![](./resources/images/4_4_knn_tuning.png "")

The models trained with the distance weighting function strongly overfit the data. It's probably due to the classes imbalance and we can safely exclude them. 

The number of neighbors has a direct impact on the bias/variance trade-off, low k values lead to highly flexible models with low bias but high variance, so more overfitting. Reciproquely, high k values lead to rigid model with high bias but low variance, so less overfitting. 

Given the results, unfortunately, all models have validation standard deviation higher than 1%.  
Our wish would be to find a model satisfying the following conditions:

* validation standard deviation <= 1.5%
* weight function = uniform
* mean validation precision >= 82%
* delta score <= 4%

Only one model fullfill all theses conditions : 

![](./resources/images/4_4_knn_best_models.png "")

By training the model on the full training set and evaluating it on the test set, we obtain the following results:

![](./resources/images/4_4_knn_results.png "")

The training and test precision scores are close indicating that the model generalizes well the data. The test validation score is even better than predicted by cross-validation.

This model is clearly better than the baseline models and the decision tree but has lower precision than the random forest model, but higher recall. With a precision score of 86% and a recall of 90%, this knn model appears to be a well balanced, intuitive and performing model.

Compared to the random forest model, it has predicted 20 True Positives and 14 False Positives more.

## 4.5 Logistic Regression

The data and plots presented in this section come from the `4_5_Logistic_Regression.ipynb` notebook.

A logistic regression model is a probabilistic classifier that gives the probabilities than a test point belongs to one class or the other. A binary classifier could be built simply by fitting a linear regression. However, we would have to deal with negative and greater than one probabilities. So, instead of relating probabilities to a linear function in the features space, logistic regression use a logistic function (sigmoid curve), a strongly non-linear function that guarantee that the probabilities remains in the [0, 1] range. How this probabilistic function relates to the features ? In fact, by the way the logistic model is expressed, its logit/log-odds is linear in the feature space X. Who say linear regression, say that a vector of p+1 coefficients has to be learned where p is the number of features. It will be the job of the SGD algorithm to find these coefficients. 

The following hyperparameters have been tuned by grid search and 5-folds cross-validation:

* PCA number of components
* Regularizazion strength $\alpha$
* Regularization term: L1 and L2

Note that by reducing the number of dimensions with PCA, we also reduce the number of coefficients the model has to learn. The regularizazion strength allow us to manage the bias-variance trade-off: high alpha => high bias/low variance, low alpha => low bias/high variance.

Because the imbalance of our target variable, we apply weights inversely proportional to class frequencies: `class_weight='balanced'`.

1000 models have been trained, here are the results presented as three scatter plots:

![](./resources/images/4_5_logistic_tuning.png "")

There are models with a validation standard deviation below 1%. The models with higher regularization strength have less spreaded validation standard deviation range because they tend to have higher bias but lower variance. Given the fact that our problem isn't linear, the impact on the bias is less evident...

Given the results, our wish would be to find a model satisfying the following conditions:

* validation standard deviation <= 1%
* mean validation precision >= 75%

![](./resources/images/4_5_logistic_best_models.png "")

The mean validation precision score being the same, selecting the model with the best validation standard deviation sounds indicated...

By training the model on the full training set and evaluating it on the test set, we obtain the following results:

![](./resources/images/4_5_logistic_results.png "")

Again, the test precision score is clearly better than predicted by cross-validation. The test dataset doesn't have a lot of samples, hence the observed test score variance.

The test precision score is better than the baseline models but worse than the other kind of models. However, it does a good job to identify most of the exoplanets with a recall at 90%.

## 4.6 Linear Support Vector Machine

The data and plots presented in this section come from the `4_6_SVM_Linear.ipynb` notebook.

Linear support vector machine try to find an optimal hyperplane with margins to separate two classes. In a nutshell, SVC try to find the best hyperplane that classify correctly the observations and in the same time try to maximize their distance (the margins) from this plane. Small margins allow to have models with low bias (few misclassifications) but because there are also few support vectors, with high variance. Reciproquely, bigger margins will allow more misclassifications (higher bias) but with margins defined by more support vectors, so less flexible, so a model with less variance. The C parameter in `LinearSVC` allows to tune this bias/variance trade-off: small C value lead to bigger margins (high bias/low variance, potential underfitting), high C values lead to smaller margins (low bias/high variance, potential overfitting). For our model, we will use the `LinearSVC` algorithm that is computationally more efficient than `SVC` with a linear kernel.

The following hyperparameters have been tuned by grid search and 5-folds cross-validation:

* PCA number of components
* Penalty parameter C of the error term
* Loss function
 
Note that because the imbalance of our target variable, we apply weights inversely proportional to class frequencies: `class_weight='balanced'`.

1000 models have been trained, here are the results presented as three scatter plots:

![](./resources/images/4_6_svm_linear_tuning.png "")

As expected, when C is too low, the bias increase and the precision score drop. It appears that models with C > 0.1 and hinge loss function are better.

Given the results, our wish would be to find a model satisfying the following conditions:

* validation standard deviation <= 1%
* mean validation precision >= 75%

![](./resources/images/4_6_svm_linear_best_models.png "")

The models with the best validation standard deviation has also the best mean validation precision score. So, we will select this first model as our best linear SVM model. Note that all the models are models for which the PCA number of components is set to None (=21). It can be probably explained by the fact that more dimensions give more flexibility to find an optimal hyperplane in the N dimensional feature space.

By training the model on the full training set and evaluating it on the test set, we obtain the following results:

![](./resources/images/4_6_svm_linear_results.png "")

Given the non linear nature of our problem, it's not surprising that this model performs less well than the others models regarding the precision metric. It appears that linear models are good to identify most of the exoplanets, represented here by a recall of 93%. The test precision score of 80% is better than predicted by cross-validation but probably by chance.

We expect the SVM model with an RBF kernel to be much better than this model and probably better also than the random forest and knn models...

## 4.7 Support Vector Machine with RBF Kernel

The data and plots presented in this section come from the `4_7_SVM_RBF.ipynb` notebook.

The `SVC` classifier allows to use different kernels. RBK kernel add a new gamma hyperparameter to tune with grid search. The gamma parameter controls how much the points affect each other. Larger is the gamma value, closer the data points should be to produce significant kernel values. The gamma parameters can be seen as the inverse of the radius of influence of samples selected by the model as support vectors.

The following hyperparameters have been tuned by grid search and 5-folds cross-validation:

* PCA number of components
* Penalty parameter C of the error term
* RBF kernel coefficient $\gamma$
 
Note that because the imbalance of our target variable, we apply weights inversely proportional to class frequencies: `class_weight='balanced'`.

350 models have been trained, here are the results presented as three scatter plots:

![](./resources/images/4_7_svm_rbf_tuning.png "")

Models with high C values quickly overfit the data, only small gamma value allow to prevent it. The best models, the red data points, have a C value below 0.5 and a gamma above 0.1.

Given the results, our wish would be to find a model satisfying the following conditions:

* validation standard deviation <= 1%
* delta mean precision score <= 2.5%
* validation precision score >= 90%

![](./resources/images/4_7_svm_rbf_best_models.png "")

The first model appears to be a good choice, it has the best validation standard deviation and mean validation precision score.

By training the model on the full training set and evaluating it on the test set, we obtain the following results:

![](./resources/images/4_7_svm_rbf_results.png "")

The training and test precision scores are the same, at 91%, indicating that our model don't overfit the data.
This SVM with RBF kernel model is our best model in regards of the precision metric and it was quiet expected.

However, it's interesting to note that its recall is very bad compared to the other models, meaning that it miss a huge proportion of the exoplanets. It is a more opiniated precision focused model. Comparison with our second best precision model, the random forest model, it improves the precision by 2% at the cost of recall 38% lower...

# 5. Results and Candidates Assessment

## 5.1 Model Results Summary

The data and plots presented in this section come from the `5_Results.ipynb` notebook.


Here is a summary of the results obtained on the **test set** for each of our best models :

![](./resources/images/5_1_model_results.png "")

Nothing new but it can be easier to interpret the data with the following scatter plots :

* Precision VS Recall
* Number of TP VS Number of FP

![](./resources/images/5_1_model_plots.png "")

The SVM RBF model appears as an outlier and the cost to pay to improve the precision by 2% feels expensive but I think it would be wrong to change our mind now and to favor a more balanced model. If we would like to have the best balanced model, we should tune our models to optimize the F1 Score. This bad feeling explain probably why the F1 Score is often choosen to tune models.

In our case, the precision metric choice was motivated by a scenario in which resources like time and budget are constrained. In such a case, we don't care to miss some positive cases because we will never have the resources to investigate about all of them. The important thing is that the resources are invested as much as possible to positive cases and nothing else. The precision metric is all about that and we have to accept the resulting recall.

A maybe better way to know if this trade-off is acceptable is to put it in front of the use case. Let's say that we have estimated that our time/budget resources will allow us to investigate about 200 exoplanet candidates among the 1'000 in the candidate list. So, to decide we have to:

1. Assess the candidates with our best models
2. Count the number of candidates that have been classified as strong candidates (CONFIRMED) by our best model
3. Conclude :
    * if our SVM RBF model predict that less than 200 candidates are strong candidates, so it can means that it is too much constrained by the precision metric. We could then decide to use the random forest model instead or use it to get the missing candidates
    * if our SVM RBF model predict that more than 200 candidates are strong candidates, it means that we would be happy to have an even more precise model at the cost of an even worse recall.
    
All of that to say that this tradeoff can be acceptable depending our needs. 

So, let's now assess the candidates and next we will experiment with the `predict_proba()` method, the operational probability threshold and the precision-recall curve.

## 5.2 Candidates Assessment

After dropping the candidates outside the range defined by the threshold indicator variables engineered during the EDA, it remains **823 candidates**. 

We could push deeper the first selection of the candidates with a specific EDA and some insights we get during the EDA, i.e:

* the analysis of the maximum multi-event statistic VS signal-to-noise ration could allow to reject some candidates
* there are more probabilities to find exoplanets around star hosting many TCEs (potential planetery system)
* etc.

We will not do it because it would not demonstrate more knowledge than during the EDA but it should be done...

Here are the number of strong candidates (aka confirmed exoplanets) predicted by our models when assessing the candidates list :

![](./resources/images/5_2_candidates_assessment.png "")

According to our models and their precision metric:

* The SVM RBF model says that 30% of the candidates are strong candidates, 254 in all, and given its precision score we can expect that 91% of them are really exoplanets and 9% are misclassified exoplanets
* The Random Forest model, our second best model, says that 70% of the candidates are strong candidates, 582 in all. We can expect 89% of them to be exoplanets and 11% to be misclassified exoplanets

If we have time/budget to study less than 254 candidates there is no reason to choose an other model than the SVM RBF model, but what if we have resources to only study, let's say 10 candidates ?

It would be better to know the probability that a candidate is an exoplanet and choose the top 10 candidates with the highest probabilities, right ?

## 5.3 Optional experimentation: predict probabilities, operational threshold, precision-recall curve

Except the logistic regression model that is a probablistic model by nature, all other models we have trained aren't. However, we can estimate the probabilities by using some "tricks" and the estimators offer a `predict_proba()` method just for that.

Here are the 10 candidates with the highest probabilites according to the SVM RBF model :

![](./resources/images/5_3_candidates_probs.png "")

The meaning of these probabilities are not all equal given the models :

* Decision tree: the predicted class probability is the fraction of samples of the same class in a leaf
* Random forest: the predicted class probabilities of an input sample are computed as the mean predicted class probabilities of the trees in the forest
* k-NN: the predicted class probability is the ratio of the neighbors with the positive class to the k hyperparameter
* SVMs: the predicted class probabilities can only be estimated by training a model with cross-validation on top (wrapping) of the initial pipeline training

Knowing these probabilities, we could now decide that a strong candidate is a candidate with a probability greater than 80% for example. This threshold is called operational threshold. And for each threshold, we can compute the contingency matrix and the derived metrics.

It's common to plot the model precision VS threshold, recall VS threshold and the precision-recall curves. They give better insights about the model behaviour, capabilities and weaknesses.

To conclude this project, here are these plots for the logistic regression, random forest and SVM with RBF kernel models :

![](./resources/images/5_3_precision_recall_curves.png "")

I didn't expected these results and it ask some questions I will need to answer and study on my own after this project.

First of all, the logistic regression model beeing a probabilistic classifier, we can see that its precision metric is much better if we consider an operational threshold near 90% instead the tacit 50% used to tuned and select the model. 

By extension, it means that it probably make no sense to draw this plots now and even less to compare the curves of each models. This job should have been done during the grid search by generating these curves for each models and find the models with the best precision metric and its associated operational threshold.

Given these plots, the SVM RBF model is clearly not the best model but we have to remember that the probabilities, except for the logistic regression models are estimated using some tricks. For the SVM models, the predicted probabilities can even be contradictory with the predicted class. 

- Does it make sense to really use the `predict_proba()` method with model that aren't probabilistic classifier ?
- I experimented with the ROC area under curve metric but for some models like k-NN and random forest, this metric tends to favor high k value or a large number of trees because the bad discretization of the resulting ROC curve if not the case. Sounds not good when the quality of the measure is function of the hyperparameters we want to tune... In such a situation, how to select the model with the best ROC curve or precission-recall curve and be able to scale when we have more than 1'000 models trained by grid-search ?

Overall, I have follow the same methodology learnt during the course but with the precision metric instead of the accuracy metric. So, it should not be wrong but I could have probably find a better model by tuning the operational threshold at the grid-search stage. 

It's presented here as an optional experimentation and an opportunity to discuss it after the defense.

# 6. Final Words

It was a long journey to achieve this capstone project and an opportunity to practice a lot of material learnt during the course. In this regards, the goal is achieved and each major steps of a machine learning project have been done:

1. Collect and prepare the data for analysis
2. Data cleaning, handling of the missing values
3. Exploratory Data Analysis
4. Feature engineering
5. Make a machine learning plan, discussing and choosing a metric adapted to the business domain/use case
6. Training different kinds of models with grid-search and cross-validation
7. Selecting and training the best model on the full training set and evaluating it on the test set
8. Get predictions by using the model on unlabeled data

In this project, I make the choice to predict if a TCE is an exoplanet and use the precision metric to identify the most promising candidates. I tend to think it would have been better to build classifiers that predict if a TCE is a false positive and use the recall metric to try to identify most of them, assess the candidates and select the candidates not rejected as false positives as the most promising candidates.